# 문제: 제조 라인 이상(불량) 탐지 프로젝트

## 배경

제조 라인에서는 수백 개의 공정 변수(설비, 품질, 환경 등)가 복합적으로 작용하며, 공정 내 미세한 이상 조짐이 누적되어 최종 불량(EOL Fail)이 발생한다. 주어진 데이터셋은 배치(batch) 단위의 주요 공정 특성치를 담고 있으며, 타깃 변수 `eol_fail`은 해당 배치의 최종 불량 여부(0: 정상, 1: 불량)를 의미한다.

이번 과제의 목표는 지도 학습과 비지도 학습을 결합하여 이상 발생 위험도를 확률(score) 형태로 산출하고, 운영 현장에 적용 가능한 기준(Threshold Policy)을 제시하는 것이다.

---

## 1️⃣ 데이터 로딩 및 탐색

* CSV 파일: `manufacturing_error_dataset_v2.csv`
* 타깃 컬럼: `eol_fail`
* 주요 범주형 컬럼: `line_id`, `shift`, `supplier`, `material_grade`
* 수치형 컬럼: 그 외 전부
* 데이터 크기와 클래스 비율을 출력하고, 불균형 여부를 확인하시오.

---

## 2️⃣ 피처 추출 및 전처리

* 범주형과 수치형 컬럼을 분리하고, 적절한 전처리를 수행하시오.
* 비지도 학습을 활용하여 잠재 특징(latent features)을 생성하고, 기존 피처와 결합하시오.
* 데이터 누락, 이상치, 불균형 등의 문제를 식별하고 해결하시오.

---

## 3️⃣ 지도 학습 모델 설계 및 학습

* 타깃 `eol_fail`을 예측하기 위한 분류 모델을 학습하시오.
* 적절한 성능 지표를 선택하고, 해당 모델을 평가하시오.
* 불균형 데이터에 대한 대응 방법을 포함하시오.

---

## 4️⃣ 임계값 탐색 및 이상 판정

* 검증 세트의 예측 확률(`y_prob`)을 활용하여 다음을 수행하시오:

  1. **최적 임계값** 탐색 (예: F1 기준)
  2. **Top 5% 기준**으로 이상을 판정
* 두 기준에 대한 정밀도, 재현율, F1 점수를 비교하시오.

---

## 5️⃣ 이상 샘플 리포트 출력

* 검증셋에서 예측 확률 상위 5% 샘플을 이상으로 간주하고, 아래 컬럼을 포함하여 출력하시오.

  ```
  line_id, shift, supplier, material_grade, score, eol_fail
  ```
* 상위 30건(또는 전체)을 score 내림차순으로 정렬하여 제시하시오.

---

## 제출 안내

* 본 과제는 **코드 구현 결과물만 제출**합니다.
* 제출 파일 형식: `.ipynb` (Jupyter Notebook)
* 별도의 리포트나 서술형 분석은 포함하지 않습니다.
* 실행 가능한 코드와 출력 결과가 모두 포함되어야 합니다.

---

## 평가 기준

| 구분     | 요구 수준                    | 비고 |
| ------ | ------------------------ | -- |
| 데이터 탐색 | 구조, 분포, 불균형 분석           | 필수 |
| 전처리    | 범주형/수치형 분리 및 변환          | 필수 |
| 피처 추출  | 비지도 표현 학습 또는 자체 피처 엔지니어링 | 필수 |
| 모델링    | 분류 모델 자유 선택              | 필수 |
| 임계값 탐색 | F1 최적 vs Top 5% 비교       | 필수 |
| 코드 완결성 | 오류 없이 실행 가능한 구조          | 필수 |


In [1]:
!pip install gdown

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# ========== 1) Imports ==========
import os, math, time, random, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  # <-- Compact view 시각화를 위해 추가

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import (
    roc_auc_score, average_precision_score, f1_score, precision_recall_curve
)
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)


In [4]:
!gdown --fuzzy 'https://drive.google.com/file/d/1GPFqmquPgvcLQQqSem3OPhR-hT7zStB2/view?usp=sharing' -O manufacturing_error_dataset_v2.csv

CSV_PATH = "/content/manufacturing_error_dataset_v2.csv"
assert os.path.exists(CSV_PATH), f"CSV not found: {CSV_PATH}"

Downloading...
From: https://drive.google.com/uc?id=1GPFqmquPgvcLQQqSem3OPhR-hT7zStB2
To: /content/manufacturing_error_dataset_v2.csv
100% 161k/161k [00:00<00:00, 17.4MB/s]


In [5]:

# ========== 2) Repro & Device ==========
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_CUDA = device.type == "cuda"

# ========== 3) Load Data ==========
CSV_PATH = "/content/drive/MyDrive/Colab Notebooks/manufacturing_error_dataset_v2.csv"
assert os.path.exists(CSV_PATH), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

target_col = "eol_fail"
y = df[target_col].astype(int)
cat_cols = ["line_id", "shift", "supplier", "material_grade"]
num_cols = [c for c in df.columns if c not in cat_cols + [target_col]]
X = df[cat_cols + num_cols].copy()

X_train_all, X_valid, y_train_all, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

In [6]:
X_train_all.shape, X_valid.shape, y_train_all.shape, y_valid.shape

((2000, 15), (500, 15), (2000,), (500,))

# 데이터 로딩 및 탐색
1️⃣ 데이터 로딩 및 탐색
CSV 파일: manufacturing_error_dataset_v2.csv
타깃 컬럼: eol_fail
주요 범주형 컬럼: line_id, shift, supplier, material_grade
수치형 컬럼: 그 외 전부
데이터 크기와 클래스 비율을 출력하고, 불균형 여부를 확인하시오.

In [7]:
CSV_PATH = "/content/drive/MyDrive/Colab Notebooks/manufacturing_error_dataset_v2.csv"
assert os.path.exists(CSV_PATH), f"CSV not found: {CSV_PATH}"
df = pd.read_csv(CSV_PATH)

target_col = "eol_fail"
y = df[target_col].astype(int)

# 범주형/수치형 Feature 구분
cat_cols = ["line_id", "shift", "supplier", "material_grade"]
num_cols = [c for c in df.columns if c not in cat_cols + [target_col]]

X = df[cat_cols + num_cols].copy()

X_train_all, X_valid, y_train_all, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

In [8]:
X_train_all.shape, X_valid.shape, y_train_all.shape, y_valid.shape

((2000, 15), (500, 15), (2000,), (500,))

In [9]:
# 결측치 여부 확인
print("=" * 60)
print("결측치 여부 확인")
print("=" * 60)
print(X_train_all.isnull().sum())
print(f"\n총 결측치 수: {X_train_all.isnull().sum().sum()}")

print("\n[Test 결측치 | Test Missing Values]")
print(X_valid.isnull().sum())
print(f"\n총 결측치 수: {X_valid.isnull().sum().sum()}")


결측치 여부 확인
line_id                     0
shift                       0
supplier                    0
material_grade              0
machine_age_months          0
last_maint_days             0
operator_experience_yrs     0
ambient_temp_c              0
ambient_humidity_pct        0
vibration_rms               0
hydraulic_pressure_bar      0
defect_history_count_90d    0
prev_batch_quality_score    0
tool_wear_mm                0
scrap_rate_pct_line_week    0
dtype: int64

총 결측치 수: 0

[Test 결측치 | Test Missing Values]
line_id                     0
shift                       0
supplier                    0
material_grade              0
machine_age_months          0
last_maint_days             0
operator_experience_yrs     0
ambient_temp_c              0
ambient_humidity_pct        0
vibration_rms               0
hydraulic_pressure_bar      0
defect_history_count_90d    0
prev_batch_quality_score    0
tool_wear_mm                0
scrap_rate_pct_line_week    0
dtype: int64

총 결측치 수: 0


# 피처 추출 및 전처리
2️⃣ 피처 추출 및 전처리
범주형과 수치형 컬럼을 분리하고, 적절한 전처리를 수행하시오.
비지도 학습을 활용하여 잠재 특징(latent features)을 생성하고, 기존 피처와 결합하시오.
데이터 누락, 이상치, 불균형 등의 문제를 식별하고 해결하시오.

In [10]:
# get_dummies() 함수를 이용한 Categorical feature 전처리 진행
X_train_cat = pd.get_dummies(X_train_all[cat_cols], columns=cat_cols, dtype=int)
X_valid_cat = pd.get_dummies(X_valid[cat_cols], columns=cat_cols, dtype=int)

In [11]:
X_train_cat

,line_id_L1,line_id_L2,line_id_L3,line_id_L4,line_id_L5,line_id_L6,shift_A,shift_B,shift_C,supplier_S1,supplier_S2,supplier_S3,supplier_S4,material_grade_M_A,material_grade_M_B,material_grade_M_C
1497,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0
2106,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0
247,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0
2320,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0
2091,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
2104,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0
2010,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0
2108,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1


In [12]:
X_valid_cat

,line_id_L1,line_id_L2,line_id_L3,line_id_L4,line_id_L5,line_id_L6,shift_A,shift_B,shift_C,supplier_S1,supplier_S2,supplier_S3,supplier_S4,material_grade_M_A,material_grade_M_B,material_grade_M_C
1126,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0
80,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0
2382,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0
1250,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0
1313,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0
184,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0
1598,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0
2085,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0


In [13]:
# 수치형 데이터 스케일링
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()

# 선택한 특성들에 대해 한 번에 fit_transform 적용
# 이 과정은 NumPy 배열을 반환하므로 다시 DataFrame으로 변환할 수 있습니다.
arr_scaled_X_train_numeric = scaler.fit_transform(X_train_all[num_cols])
arr_scaled_X_valid_numeric = scaler.transform(X_valid[num_cols])

# 변환된 데이터를 다시 DataFrame으로 만들기
scaled_X_train_numeric = pd.DataFrame(arr_scaled_X_train_numeric, columns=num_cols, index=X_train_all.index)
scaled_X_valid_numeric = pd.DataFrame(arr_scaled_X_valid_numeric, columns=num_cols, index=X_valid.index)


In [14]:
scaled_X_train_numeric

,machine_age_months,last_maint_days,operator_experience_yrs,ambient_temp_c,ambient_humidity_pct,vibration_rms,hydraulic_pressure_bar,defect_history_count_90d,prev_batch_quality_score,tool_wear_mm,scrap_rate_pct_line_week
1497,1.666410,-0.288216,-0.524957,0.338864,0.127950,0.068555,-0.486715,-0.166071,-0.312819,1.159599,0.073837
2106,-1.466303,-0.432548,-0.964142,0.383580,-1.485691,0.947748,1.220791,0.726785,-0.969506,-1.020968,-0.313379
247,0.970251,-1.298539,1.758802,-1.002597,0.144672,1.610448,1.085543,0.726785,-0.661684,-0.353098,0.792952
2320,-0.402728,0.866438,-0.554236,1.210814,-0.833546,0.620529,1.077090,-1.058927,1.021076,1.205266,0.219043
2091,0.409457,1.010770,-1.374047,-0.153005,0.077785,-2.578940,0.874218,0.726785,-0.764291,1.462139,0.786038
...,...,...,...,...,...,...,...,...,...,...,...
862,0.467470,1.068503,0.851154,-0.197720,0.136311,0.594087,-0.097877,0.726785,1.657242,1.644804,0.122239
2104,-1.060210,-0.952143,1.085386,-0.197720,-0.448947,-0.319811,-0.047159,-1.058927,-0.435948,-0.107641,0.205214
2010,0.486808,-0.605746,-0.466399,0.696587,-0.850267,-0.521430,0.510739,-1.058927,-0.846377,0.628728,-1.205358
2108,-0.789482,-0.345949,-0.261446,-1.919262,0.688126,-0.058697,-0.917818,-0.166071,0.384911,0.172065,1.608872


In [15]:
final_X_train_all = pd.concat([scaled_X_train_numeric, X_train_cat], axis=1)
final_X_valid = pd.concat([scaled_X_valid_numeric, X_valid_cat], axis=1)

In [16]:
final_X_train_all

,machine_age_months,last_maint_days,operator_experience_yrs,ambient_temp_c,ambient_humidity_pct,vibration_rms,hydraulic_pressure_bar,defect_history_count_90d,prev_batch_quality_score,tool_wear_mm,...,shift_A,shift_B,shift_C,supplier_S1,supplier_S2,supplier_S3,supplier_S4,material_grade_M_A,material_grade_M_B,material_grade_M_C
1497,1.666410,-0.288216,-0.524957,0.338864,0.127950,0.068555,-0.486715,-0.166071,-0.312819,1.159599,...,1,0,0,1,0,0,0,0,1,0
2106,-1.466303,-0.432548,-0.964142,0.383580,-1.485691,0.947748,1.220791,0.726785,-0.969506,-1.020968,...,0,1,0,0,1,0,0,0,1,0
247,0.970251,-1.298539,1.758802,-1.002597,0.144672,1.610448,1.085543,0.726785,-0.661684,-0.353098,...,0,1,0,0,1,0,0,1,0,0
2320,-0.402728,0.866438,-0.554236,1.210814,-0.833546,0.620529,1.077090,-1.058927,1.021076,1.205266,...,0,0,1,0,0,0,1,0,1,0
2091,0.409457,1.010770,-1.374047,-0.153005,0.077785,-2.578940,0.874218,0.726785,-0.764291,1.462139,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,0.467470,1.068503,0.851154,-0.197720,0.136311,0.594087,-0.097877,0.726785,1.657242,1.644804,...,1,0,0,0,0,1,0,0,1,0
2104,-1.060210,-0.952143,1.085386,-0.197720,-0.448947,-0.319811,-0.047159,-1.058927,-0.435948,-0.107641,...,0,1,0,1,0,0,0,0,1,0
2010,0.486808,-0.605746,-0.466399,0.696587,-0.850267,-0.521430,0.510739,-1.058927,-0.846377,0.628728,...,1,0,0,1,0,0,0,1,0,0
2108,-0.789482,-0.345949,-0.261446,-1.919262,0.688126,-0.058697,-0.917818,-0.166071,0.384911,0.172065,...,0,1,0,0,1,0,0,0,0,1


In [17]:
final_X_valid

,machine_age_months,last_maint_days,operator_experience_yrs,ambient_temp_c,ambient_humidity_pct,vibration_rms,hydraulic_pressure_bar,defect_history_count_90d,prev_batch_quality_score,tool_wear_mm,...,shift_A,shift_B,shift_C,supplier_S1,supplier_S2,supplier_S3,supplier_S4,material_grade_M_A,material_grade_M_B,material_grade_M_C
1126,1.453695,0.722107,-1.461884,0.316507,-0.858628,-0.802375,0.214884,-0.166071,-1.769843,1.091100,...,0,0,1,1,0,0,0,1,0,0
80,1.086278,1.559231,0.411970,1.680325,-1.377000,-1.641905,0.713611,-0.166071,-1.010549,0.834227,...,0,0,1,0,1,0,0,1,0,0
2382,1.086278,0.260245,1.583128,-1.785116,0.813538,1.405523,-1.255939,-0.166071,-0.415426,0.097857,...,0,1,0,0,0,1,0,1,0,0
1250,1.279655,-1.154207,0.851154,-1.718043,-1.519134,1.245219,0.181072,-1.058927,0.795340,1.650512,...,1,0,0,1,0,0,0,0,1,0
1313,0.003365,0.664374,1.583128,-0.242436,-0.657968,-0.250401,0.400850,0.726785,-1.749321,0.172065,...,0,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,1.028265,0.866438,1.173223,0.964879,0.688126,-0.299979,0.933389,1.619640,1.657242,1.319431,...,0,0,1,0,1,0,0,1,0,0
184,-0.460741,-1.154207,0.089901,0.182360,1.089446,-0.731313,0.629081,-0.166071,-1.318371,-0.729845,...,1,0,0,0,1,0,0,0,1,0
1598,0.080716,-0.374815,-1.579000,-0.242436,0.437301,-0.518125,-1.594059,-0.166071,-0.948984,1.399347,...,0,0,1,0,0,1,0,1,0,0
2085,-1.543654,0.779839,-0.378562,-0.354224,-0.816824,-0.721397,0.020465,0.726785,0.282304,-0.775511,...,1,0,0,1,0,0,0,0,1,0


In [18]:
import pandas as pd
import numpy as np

def manage_outliers_iqr(df, columns, remove=False):
    """
    데이터프레임의 지정된 수치형 컬럼들에서 IQR 방식을 사용하여 이상치를 탐지하고
    선택적으로 제거합니다.

    Args:
        df (pd.DataFrame): 원본 데이터프레임.
        columns (list): 이상치를 확인할 수치형 컬럼 이름 리스트.
        remove (bool, optional): True이면 이상치가 포함된 행을 제거한 데이터프레임을 반환합니다.
                                  False이면 이상치 정보만 출력하고 원본 데이터프레임을 반환합니다.
                                  기본값은 False입니다.

    Returns:
        pd.DataFrame: remove 매개변수에 따라 이상치가 제거된 데이터프레임 또는 원본 데이터프레임.
    """
    df_cleaned = df.copy()
    outlier_indices = set()

    print("--- 이상치 탐지 결과 (IQR 방식, 1.5 * IQR 기준) ---")

    for col in columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1

            # 이상치 경계 계산
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # 이상치 인덱스 찾기
            col_outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
            outlier_indices.update(col_outliers)

            print(f"컬럼 '{col}': 이상치 {len(col_outliers)}개 발견 (하한: {lower_bound:.2f}, 상한: {upper_bound:.2f})")
        else:
            print(f"컬럼 '{col}'은(는) 수치형이 아니므로 건너뜁니다.")

    if remove:
        # 누적된 이상치 인덱스를 사용하여 행 제거
        original_count = len(df)
        df_cleaned = df_cleaned.drop(outlier_indices)
        cleaned_count = len(df_cleaned)
        print(f"\n--- 이상치 제거 결과 ---")
        print(f"원본 데이터 행 수: {original_count}")
        print(f"제거된 이상치 행 수: {original_count - cleaned_count}")
        print(f"이상치 제거 후 데이터 행 수: {cleaned_count}")
        return df_cleaned
    else:
        print("\n* remove=False로 설정되어 이상치가 제거되지 않았습니다. 원본 데이터프레임을 반환합니다.")
        return df

In [19]:

final_X_train_all = pd.concat([final_X_train_all, y_train_all], axis=1)
final_X_valid = pd.concat([final_X_valid, y_valid], axis=1)

final_X_train_all_cleaned = manage_outliers_iqr(final_X_train_all, columns=num_cols, remove=True)
final_X_valid_cleaned = manage_outliers_iqr(final_X_valid, columns=num_cols, remove=True)

--- 이상치 탐지 결과 (IQR 방식, 1.5 * IQR 기준) ---
컬럼 'machine_age_months': 이상치 0개 발견 (하한: -3.50, 상한: 3.46)
컬럼 'last_maint_days': 이상치 0개 발견 (하한: -3.46, 상한: 3.46)
컬럼 'operator_experience_yrs': 이상치 0개 발견 (하한: -3.47, 상한: 3.44)
컬럼 'ambient_temp_c': 이상치 8개 발견 (하한: -2.68, 상한: 2.69)
컬럼 'ambient_humidity_pct': 이상치 12개 발견 (하한: -2.59, 상한: 2.60)
컬럼 'vibration_rms': 이상치 7개 발견 (하한: -2.78, 상한: 2.77)
컬럼 'hydraulic_pressure_bar': 이상치 14개 발견 (하한: -2.65, 상한: 2.66)
컬럼 'defect_history_count_90d': 이상치 4개 발견 (하한: -3.74, 상한: 3.41)
컬럼 'prev_batch_quality_score': 이상치 6개 발견 (하한: -2.87, 상한: 2.96)
컬럼 'tool_wear_mm': 이상치 9개 발견 (하한: -2.74, 상한: 2.68)
컬럼 'scrap_rate_pct_line_week': 이상치 8개 발견 (하한: -2.77, 상한: 2.76)

--- 이상치 제거 결과 ---
원본 데이터 행 수: 2000
제거된 이상치 행 수: 67
이상치 제거 후 데이터 행 수: 1933
--- 이상치 탐지 결과 (IQR 방식, 1.5 * IQR 기준) ---
컬럼 'machine_age_months': 이상치 0개 발견 (하한: -3.37, 상한: 3.46)
컬럼 'last_maint_days': 이상치 0개 발견 (하한: -3.36, 상한: 3.22)
컬럼 'operator_experience_yrs': 이상치 0개 발견 (하한: -3.57, 상한: 3.37)
컬럼 'ambient_temp_c': 이상치 2개 발견

In [20]:
final_X_train_all_cleaned

,machine_age_months,last_maint_days,operator_experience_yrs,ambient_temp_c,ambient_humidity_pct,vibration_rms,hydraulic_pressure_bar,defect_history_count_90d,prev_batch_quality_score,tool_wear_mm,...,shift_B,shift_C,supplier_S1,supplier_S2,supplier_S3,supplier_S4,material_grade_M_A,material_grade_M_B,material_grade_M_C,eol_fail
1497,1.666410,-0.288216,-0.524957,0.338864,0.127950,0.068555,-0.486715,-0.166071,-0.312819,1.159599,...,0,0,1,0,0,0,0,1,0,0
2106,-1.466303,-0.432548,-0.964142,0.383580,-1.485691,0.947748,1.220791,0.726785,-0.969506,-1.020968,...,1,0,0,1,0,0,0,1,0,0
247,0.970251,-1.298539,1.758802,-1.002597,0.144672,1.610448,1.085543,0.726785,-0.661684,-0.353098,...,1,0,0,1,0,0,1,0,0,0
2320,-0.402728,0.866438,-0.554236,1.210814,-0.833546,0.620529,1.077090,-1.058927,1.021076,1.205266,...,0,1,0,0,0,1,0,1,0,1
2091,0.409457,1.010770,-1.374047,-0.153005,0.077785,-2.578940,0.874218,0.726785,-0.764291,1.462139,...,0,1,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,0.467470,1.068503,0.851154,-0.197720,0.136311,0.594087,-0.097877,0.726785,1.657242,1.644804,...,0,0,0,0,1,0,0,1,0,1
2104,-1.060210,-0.952143,1.085386,-0.197720,-0.448947,-0.319811,-0.047159,-1.058927,-0.435948,-0.107641,...,1,0,1,0,0,0,0,1,0,0
2010,0.486808,-0.605746,-0.466399,0.696587,-0.850267,-0.521430,0.510739,-1.058927,-0.846377,0.628728,...,0,0,1,0,0,0,1,0,0,0
2108,-0.789482,-0.345949,-0.261446,-1.919262,0.688126,-0.058697,-0.917818,-0.166071,0.384911,0.172065,...,1,0,0,1,0,0,0,0,1,0


In [21]:
final_X_valid_cleaned

,machine_age_months,last_maint_days,operator_experience_yrs,ambient_temp_c,ambient_humidity_pct,vibration_rms,hydraulic_pressure_bar,defect_history_count_90d,prev_batch_quality_score,tool_wear_mm,...,shift_B,shift_C,supplier_S1,supplier_S2,supplier_S3,supplier_S4,material_grade_M_A,material_grade_M_B,material_grade_M_C,eol_fail
1126,1.453695,0.722107,-1.461884,0.316507,-0.858628,-0.802375,0.214884,-0.166071,-1.769843,1.091100,...,0,1,1,0,0,0,1,0,0,0
80,1.086278,1.559231,0.411970,1.680325,-1.377000,-1.641905,0.713611,-0.166071,-1.010549,0.834227,...,0,1,0,1,0,0,1,0,0,1
2382,1.086278,0.260245,1.583128,-1.785116,0.813538,1.405523,-1.255939,-0.166071,-0.415426,0.097857,...,1,0,0,0,1,0,1,0,0,0
1250,1.279655,-1.154207,0.851154,-1.718043,-1.519134,1.245219,0.181072,-1.058927,0.795340,1.650512,...,0,0,1,0,0,0,0,1,0,0
1313,0.003365,0.664374,1.583128,-0.242436,-0.657968,-0.250401,0.400850,0.726785,-1.749321,0.172065,...,1,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,1.028265,0.866438,1.173223,0.964879,0.688126,-0.299979,0.933389,1.619640,1.657242,1.319431,...,0,1,0,1,0,0,1,0,0,0
184,-0.460741,-1.154207,0.089901,0.182360,1.089446,-0.731313,0.629081,-0.166071,-1.318371,-0.729845,...,0,0,0,1,0,0,0,1,0,0
1598,0.080716,-0.374815,-1.579000,-0.242436,0.437301,-0.518125,-1.594059,-0.166071,-0.948984,1.399347,...,0,1,0,0,1,0,1,0,0,0
2085,-1.543654,0.779839,-0.378562,-0.354224,-0.816824,-0.721397,0.020465,0.726785,0.282304,-0.775511,...,0,0,1,0,0,0,0,1,0,0


In [22]:
# 클래스 불균형 확인
import pandas as pd
from collections import Counter

def identify_data_imbalance(df: pd.DataFrame, target_column: str) -> dict:
    """
    데이터프레임의 지정된 타겟 열에서 데이터 불균형을 식별합니다.

    Args:
        df: 불균형을 확인할 Pandas DataFrame.
        target_column: 불균형을 확인할 타겟 열의 이름 (문자열).

    Returns:
        클래스별 빈도수, 백분율 및 불균형 정도에 대한 설명을 포함하는 딕셔너리.
    """
    if target_column not in df.columns:
        return {"Error": f"'{target_column}' 열이 데이터프레임에 존재하지 않습니다."}

    # 클래스별 빈도수 계산
    counts = df[target_column].value_counts()
    total_samples = len(df)

    # 클래스별 백분율 계산
    percentages = (counts / total_samples) * 100

    # 결과 포맷팅
    results = {
        "총 샘플 수": total_samples,
        "클래스별 빈도수": counts.to_dict(),
        "클래스별 백분율 (%)": {k: round(v, 2) for k, v in percentages.to_dict().items()},
        "설명": "클래스 간 백분율 차이가 클수록(일반적으로 20~30% 이상 차이) 불균형이 심각하다고 판단할 수 있습니다."
    }

    # 불균형 지수 (간단한 예시: 가장 큰 비율 / 가장 작은 비율)
    if len(percentages) > 1:
        imbalance_ratio = percentages.max() / percentages.min()
        results["불균형 비율 (최대/최소)"] = round(imbalance_ratio, 2)
        if imbalance_ratio > 2:
             results["불균형 심각도"] = "중간 또는 높음 (리샘플링 고려 필요)"
        else:
             results["불균형 심각도"] = "낮음"
    else:
        results["불균형 심각도"] = "클래스가 하나만 존재합니다."

    return results


imbalance_info = identify_data_imbalance(final_X_train_all_cleaned, 'eol_fail')

# 결과 출력
print("--- 불균형 분석 결과 ---")
import json
print(json.dumps(imbalance_info, indent=4, ensure_ascii=False))

--- 불균형 분석 결과 ---
{
    "총 샘플 수": 1933,
    "클래스별 빈도수": {
        "0": 1648,
        "1": 285
    },
    "클래스별 백분율 (%)": {
        "0": 85.26,
        "1": 14.74
    },
    "설명": "클래스 간 백분율 차이가 클수록(일반적으로 20~30% 이상 차이) 불균형이 심각하다고 판단할 수 있습니다.",
    "불균형 비율 (최대/최소)": 5.78,
    "불균형 심각도": "중간 또는 높음 (리샘플링 고려 필요)"
}


In [23]:
y_train_all = final_X_train_all_cleaned["eol_fail"]
y_valid = final_X_valid_cleaned["eol_fail"]

X_train_all = final_X_train_all_cleaned.drop(["eol_fail"], axis=1)
X_valid = final_X_valid_cleaned.drop(["eol_fail"], axis=1)

In [24]:
y_train_all

,eol_fail
1497,0
2106,0
247,0
2320,1
2091,0
...,...
862,1
2104,0
2010,0
2108,0


In [25]:
y_valid

,eol_fail
1126,0
80,1
2382,0
1250,0
1313,0
...,...
486,0
184,0
1598,0
2085,0


In [26]:
X_train_all

,machine_age_months,last_maint_days,operator_experience_yrs,ambient_temp_c,ambient_humidity_pct,vibration_rms,hydraulic_pressure_bar,defect_history_count_90d,prev_batch_quality_score,tool_wear_mm,...,shift_A,shift_B,shift_C,supplier_S1,supplier_S2,supplier_S3,supplier_S4,material_grade_M_A,material_grade_M_B,material_grade_M_C
1497,1.666410,-0.288216,-0.524957,0.338864,0.127950,0.068555,-0.486715,-0.166071,-0.312819,1.159599,...,1,0,0,1,0,0,0,0,1,0
2106,-1.466303,-0.432548,-0.964142,0.383580,-1.485691,0.947748,1.220791,0.726785,-0.969506,-1.020968,...,0,1,0,0,1,0,0,0,1,0
247,0.970251,-1.298539,1.758802,-1.002597,0.144672,1.610448,1.085543,0.726785,-0.661684,-0.353098,...,0,1,0,0,1,0,0,1,0,0
2320,-0.402728,0.866438,-0.554236,1.210814,-0.833546,0.620529,1.077090,-1.058927,1.021076,1.205266,...,0,0,1,0,0,0,1,0,1,0
2091,0.409457,1.010770,-1.374047,-0.153005,0.077785,-2.578940,0.874218,0.726785,-0.764291,1.462139,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,0.467470,1.068503,0.851154,-0.197720,0.136311,0.594087,-0.097877,0.726785,1.657242,1.644804,...,1,0,0,0,0,1,0,0,1,0
2104,-1.060210,-0.952143,1.085386,-0.197720,-0.448947,-0.319811,-0.047159,-1.058927,-0.435948,-0.107641,...,0,1,0,1,0,0,0,0,1,0
2010,0.486808,-0.605746,-0.466399,0.696587,-0.850267,-0.521430,0.510739,-1.058927,-0.846377,0.628728,...,1,0,0,1,0,0,0,1,0,0
2108,-0.789482,-0.345949,-0.261446,-1.919262,0.688126,-0.058697,-0.917818,-0.166071,0.384911,0.172065,...,0,1,0,0,1,0,0,0,0,1


In [27]:
X_valid

,machine_age_months,last_maint_days,operator_experience_yrs,ambient_temp_c,ambient_humidity_pct,vibration_rms,hydraulic_pressure_bar,defect_history_count_90d,prev_batch_quality_score,tool_wear_mm,...,shift_A,shift_B,shift_C,supplier_S1,supplier_S2,supplier_S3,supplier_S4,material_grade_M_A,material_grade_M_B,material_grade_M_C
1126,1.453695,0.722107,-1.461884,0.316507,-0.858628,-0.802375,0.214884,-0.166071,-1.769843,1.091100,...,0,0,1,1,0,0,0,1,0,0
80,1.086278,1.559231,0.411970,1.680325,-1.377000,-1.641905,0.713611,-0.166071,-1.010549,0.834227,...,0,0,1,0,1,0,0,1,0,0
2382,1.086278,0.260245,1.583128,-1.785116,0.813538,1.405523,-1.255939,-0.166071,-0.415426,0.097857,...,0,1,0,0,0,1,0,1,0,0
1250,1.279655,-1.154207,0.851154,-1.718043,-1.519134,1.245219,0.181072,-1.058927,0.795340,1.650512,...,1,0,0,1,0,0,0,0,1,0
1313,0.003365,0.664374,1.583128,-0.242436,-0.657968,-0.250401,0.400850,0.726785,-1.749321,0.172065,...,0,1,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,1.028265,0.866438,1.173223,0.964879,0.688126,-0.299979,0.933389,1.619640,1.657242,1.319431,...,0,0,1,0,1,0,0,1,0,0
184,-0.460741,-1.154207,0.089901,0.182360,1.089446,-0.731313,0.629081,-0.166071,-1.318371,-0.729845,...,1,0,0,0,1,0,0,0,1,0
1598,0.080716,-0.374815,-1.579000,-0.242436,0.437301,-0.518125,-1.594059,-0.166071,-0.948984,1.399347,...,0,0,1,0,0,1,0,1,0,0
2085,-1.543654,0.779839,-0.378562,-0.354224,-0.816824,-0.721397,0.020465,0.726785,0.282304,-0.775511,...,1,0,0,1,0,0,0,0,1,0


# 지도학습 모델 설계 및 학습
3️⃣ 지도 학습 모델 설계 및 학습
타깃 eol_fail을 예측하기 위한 분류 모델을 학습하시오.
적절한 성능 지표를 선택하고, 해당 모델을 평가하시오.
불균형 데이터에 대한 대응 방법을 포함하시오.

In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer

# XGBoost 모델 정의 및 학습
# XGBClassifier를 사용하여 이진 분류 모델을 정의합니다.
# objective='binary:logistic'은 이진 분류에 표준적인 손실 함수입니다.
model = XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42,
    use_label_encoder=False, # 최신 버전에서는 False 권장
    eval_metric='logloss', # 평가 지표 설정
    scale_pos_weight=5  # 1648/285=5.78 (scale_pos_weight 파라미터를 사용하여 소수 클래스(양성 클래스)의 오류에 더 큰 가중치를 부여)
)


#

# 모델 학습
model.fit(X_train_all, y_train_all)

# 4. 예측 수행
# 테스트 데이터셋으로 예측합니다.
y_pred_proba = model.predict_proba(X_valid)[:, 1] # 클래스 1에 속할 확률 예측
y_pred = (y_pred_proba >= 0.5).astype(int) # 확률을 0 또는 1로 변환

# 5. 모델 평가
# 정확도 및 기타 지표를 출력합니다.
accuracy = accuracy_score(y_valid, y_pred)
print(f"정확도 (Accuracy): {accuracy:.4f}")

# 상세 분류 보고서 (정밀도, 재현율, F1-score 등)
print("\n분류 보고서 (Classification Report):")
print(classification_report(y_valid, y_pred))

# 혼동 행렬 (Confusion Matrix)
print("\n혼동 행렬 (Confusion Matrix):")
print(confusion_matrix(y_valid, y_pred))


정확도 (Accuracy): 0.9360

분류 보고서 (Classification Report):
              precision    recall  f1-score   support

           0       0.98      0.95      0.96       413
           1       0.74      0.86      0.80        71

    accuracy                           0.94       484
   macro avg       0.86      0.90      0.88       484
weighted avg       0.94      0.94      0.94       484


혼동 행렬 (Confusion Matrix):
[[392  21]
 [ 10  61]]


# 임계값 탐색 및 이상 판정

4️⃣ 임계값 탐색 및 이상 판정
검증 세트의 예측 확률(y_prob)을 활용하여 다음을 수행하시오:

최적 임계값 탐색 (예: F1 기준)
Top 5% 기준으로 이상을 판정
두 기준에 대한 정밀도, 재현율, F1 점수를 비교하시오.

In [29]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve, classification_report
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# F1 기준 최적 임계값 탐색
precisions, recalls, thresholds = precision_recall_curve(y_valid, y_pred_proba)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls)
# NaN 값은 0으로 처리 (precision 또는 recall이 0인 경우)
f1_scores[np.isnan(f1_scores)] = 0

optimal_threshold_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_threshold_idx]
print(f"F1 기준 최적 임계값: {optimal_threshold:.4f}")

# 3. 상위 5% 기준 임계값 설정
# 확률을 내림차순으로 정렬하고 상위 5% 지점의 확률값을 임계값으로 사용
top_5_percent_threshold = np.percentile(y_pred_proba, 95) # 95th percentile corresponds to the top 5%
print(f"상위 5% 기준 임계값: {top_5_percent_threshold:.4f}")

# 4. 두 기준에 대한 성능 지표 비교 함수
def evaluate_threshold(y_true, y_prob, threshold):
    y_pred = (y_prob >= threshold).astype(int)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return precision, recall, f1, y_pred

# 최적 임계값 기준 성능
opt_precision, opt_recall, opt_f1, _ = evaluate_threshold(y_valid, y_pred_proba, optimal_threshold)

# 상위 5% 기준 성능
top5_precision, top5_recall, top5_f1, _ = evaluate_threshold(y_valid, y_pred_proba, top_5_percent_threshold)

# 결과 출력
print("\n--- 성능 비교 ---")
print(f"{'기준':<15} | {'정밀도 (Precision)':<18} | {'재현율 (Recall)':<18} | {'F1 점수':<10}")
print("-" * 70)
print(f"{'F1 최적 임계값':<15} | {opt_precision:<18.4f} | {opt_recall:<18.4f} | {opt_f1:<10.4f}")
print(f"{'상위 5% 기준':<15} | {top5_precision:<18.4f} | {top5_recall:<18.4f} | {top5_f1:<10.4f}")

# 추가적으로 각 기준의 분류 보고서 확인
print("\n--- F1 최적 임계값 분류 보고서 ---")
print(classification_report(y_valid, (y_pred_proba >= optimal_threshold).astype(int)))
print("\n--- 상위 5% 기준 분류 보고서 ---")
print(classification_report(y_valid, (y_pred_proba >= top_5_percent_threshold).astype(int)))


F1 기준 최적 임계값: 0.5788
상위 5% 기준 임계값: 0.9137

--- 성능 비교 ---
기준              | 정밀도 (Precision)    | 재현율 (Recall)       | F1 점수     
----------------------------------------------------------------------
F1 최적 임계값       | 0.7763             | 0.8310             | 0.8027    
상위 5% 기준        | 0.9600             | 0.3380             | 0.5000    

--- F1 최적 임계값 분류 보고서 ---
              precision    recall  f1-score   support

           0       0.97      0.96      0.96       413
           1       0.78      0.83      0.80        71

    accuracy                           0.94       484
   macro avg       0.87      0.89      0.88       484
weighted avg       0.94      0.94      0.94       484


--- 상위 5% 기준 분류 보고서 ---
              precision    recall  f1-score   support

           0       0.90      1.00      0.94       413
           1       0.96      0.34      0.50        71

    accuracy                           0.90       484
   macro avg       0.93      0.67      0.72       484
weighted 

# 이상 샘플 리포트 출력

5️⃣ 이상 샘플 리포트 출력
검증셋에서 예측 확률 상위 5% 샘플을 이상으로 간주하고, 아래 컬럼을 포함하여 출력하시오.

line_id, shift, supplier, material_grade, score, eol_fail
상위 30건(또는 전체)을 score 내림차순으로 정렬하여 제시하시오.

In [30]:
y_pred_proba[:5]

array([0.76194537, 0.20757362, 0.85781324, 0.11187711, 0.0417211 ],
      dtype=float32)

In [31]:
y_pred_proba.shape

(484,)

In [32]:
import pandas as pd
import numpy as np

def get_top_anomalies(df_validation, prob_1d_array, select_columns, top_n=30):
    # 전체 행 수의 5%에 해당하는 임계값 계산
    df_validation["y_pred_proba"] = pd.Series(prob_1d_array, index=df_validation.index)
    threshold_value = df_validation["y_pred_proba"].quantile(0.95)

    # 상위 5%에 해당하는 샘플 필터링 (확률이 높을수록 비정상일 가능성이 높다고 가정)
    anomalies_top_5_percent = df_validation[df_validation["y_pred_proba"] >= threshold_value]

    # 3. 'score'(확률) 기준 내림차순 정렬
    sorted_anomalies = anomalies_top_5_percent.sort_values(
        by="y_pred_proba",
        ascending=False
    )

    # 4. 상위 N건(30건) 선택
    top_n_anomalies = sorted_anomalies.head(top_n)

    # 5. 특정 컬럼만 포함하여 반환
    # 만약 prob_column_name이 select_columns에 포함되어 있지 않다면 추가합니다.
    final_columns = list(set(select_columns) | {"y_pred_proba"})

    return top_n_anomalies[final_columns]


selected_cols = ["line_id", "shift", "supplier", "material_grade", "y_pred_proba", "eol_fail"]

X_train_all, X_valid, y_train_all, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

final_X_valid = pd.concat([scaled_X_valid_numeric, X_valid[cat_cols], y_valid], axis=1)
final_X_valid_cleaned = manage_outliers_iqr(final_X_valid, columns=num_cols, remove=True)


top_30_anomalies_result = get_top_anomalies(
    df_validation=final_X_valid_cleaned,
    prob_1d_array=y_pred_proba,
    select_columns=selected_cols,
    top_n=30
)

print(f"전체 검증셋 크기: {len(y_train_all)} 행")
print(f"상위 5% 샘플 수 (최소): {int(len(y_train_all) * 0.05)} 행 이상")
print(f"최종 반환된 상위 {len(top_30_anomalies_result)} 행:")
print(top_30_anomalies_result)


--- 이상치 탐지 결과 (IQR 방식, 1.5 * IQR 기준) ---
컬럼 'machine_age_months': 이상치 0개 발견 (하한: -3.37, 상한: 3.46)
컬럼 'last_maint_days': 이상치 0개 발견 (하한: -3.36, 상한: 3.22)
컬럼 'operator_experience_yrs': 이상치 0개 발견 (하한: -3.57, 상한: 3.37)
컬럼 'ambient_temp_c': 이상치 2개 발견 (하한: -2.94, 상한: 2.99)
컬럼 'ambient_humidity_pct': 이상치 2개 발견 (하한: -2.97, 상한: 2.67)
컬럼 'vibration_rms': 이상치 2개 발견 (하한: -2.84, 상한: 2.82)
컬럼 'hydraulic_pressure_bar': 이상치 2개 발견 (하한: -2.62, 상한: 2.64)
컬럼 'defect_history_count_90d': 이상치 1개 발견 (하한: -3.74, 상한: 3.41)
컬럼 'prev_batch_quality_score': 이상치 5개 발견 (하한: -2.62, 상한: 2.81)
컬럼 'tool_wear_mm': 이상치 0개 발견 (하한: -2.90, 상한: 3.12)
컬럼 'scrap_rate_pct_line_week': 이상치 2개 발견 (하한: -3.02, 상한: 2.89)

--- 이상치 제거 결과 ---
원본 데이터 행 수: 500
제거된 이상치 행 수: 16
이상치 제거 후 데이터 행 수: 484
전체 검증셋 크기: 2000 행
상위 5% 샘플 수 (최소): 100 행 이상
최종 반환된 상위 25 행:
     shift line_id material_grade supplier  eol_fail  y_pred_proba
1775     A      L1            M_A       S4         1      0.994453
224      B      L5            M_A       S1         1  